In [1]:
import pandas as pd
import numpy as np
from numpy import concatenate
from pandas import concat
from pandas import read_csv
from pandas import DataFrame
import tensorflow as tf
from tensorflow import keras
import os
import datetime
import IPython
import IPython.display
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, Input, Reshape, RepeatVector, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
%matplotlib inline
from scipy.stats import randint as sp_randint
np.random.seed(123) # for reproducibility
import warnings
warnings.filterwarnings('ignore')
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner

In [2]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Giron_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi', 'Temp_Seca_Max_Dia_Prom_1',
       'Temp_Seca_Max_Dia_Prom_2','Temp_Seca_Max_Dia_Prom_3','Temp_Seca_Max_Dia_Prom_4', 
       'Temp_Seca_Max_Dia_Prom_5','Temp_Seca_Max_Dia_Prom_6', 'Temp_Seca_Max_Sem_1',
       'Temp_Seca_Max_Sem_2', 'Temp_Seca_Max_Sem_3', 'Temp_Seca_Max_Sem_4',
       'Temp_Seca_Max_Sem_5', 'Temp_Seca_Max_Sem_6','Temp_Seca_Min_Dia_Prom_1',
       'Temp_Seca_Min_Dia_Prom_2','Temp_Seca_Min_Dia_Prom_3', 'Temp_Seca_Min_Dia_Prom_4',
       'Temp_Seca_Min_Dia_Prom_5', 'Temp_Seca_Min_Dia_Prom_6','Temp_Seca_Min_Sem_1',
       'Temp_Seca_Min_Sem_2','Temp_Seca_Min_Sem_3', 'Temp_Seca_Min_Sem_4', 
       'Temp_Seca_Min_Sem_5','Temp_Seca_Min_Sem_6','Prom_Dia_Preci_1',
       'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3', 'Prom_Dia_Preci_4',
       'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6','Total_Preci_Sem_1',
       'Total_Preci_Sem_2', 'Total_Preci_Sem_3','Total_Preci_Sem_4',
       'Total_Preci_Sem_5', 'Total_Preci_Sem_6','Temp_Max_Dia_Prom_1',
       'Temp_Max_Dia_Prom_2','Temp_Max_Dia_Prom_3', 'Temp_Max_Dia_Prom_4',
       'Temp_Max_Dia_Prom_5','Temp_Max_Dia_Prom_6','Temp_Max_Sem_1','Temp_Max_Sem_2',
       'Temp_Max_Sem_3', 'Temp_Max_Sem_4', 'Temp_Max_Sem_5','Temp_Max_Sem_6',
       'Temp_Min_Dia_Prom_1','Temp_Min_Dia_Prom_2', 'Temp_Min_Dia_Prom_3',
       'Temp_Min_Dia_Prom_4','Temp_Min_Dia_Prom_5', 'Temp_Min_Dia_Prom_6',
       'Temp_Min_Sem_1', 'Temp_Min_Sem_2', 'Temp_Min_Sem_3', 'Temp_Min_Sem_4',
       'Temp_Min_Sem_5', 'Temp_Min_Sem_6','Hum_Rel_Max_1','Hum_Rel_Max_2',
       'Hum_Rel_Max_3', 'Hum_Rel_Max_4', 'Hum_Rel_Max_5','Hum_Rel_Max_6',
       'Hum_Rel_Min_1', 'Hum_Rel_Min_2','Hum_Rel_Min_3', 'Hum_Rel_Min_4',
       'Hum_Rel_Min_5', 'Hum_Rel_Min_6'],axis=1)
datos.columns

Index(['Casos_Dengue', 'Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem',
       'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
       'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
       'Temp_Min_Dia_Prom', 'Temp_Min_Sem', 'Hum_Rel_Max', 'Hum_Rel_Min'],
      dtype='object')

In [3]:
from sklearn.impute import KNNImputer
knnImputer = KNNImputer (missing_values=np.nan, n_neighbors=2, 
                         weights="uniform", metric="nan_euclidean")
# Ajustamos el modelo e imputamos los missing values

knnImputer.fit(datos[['Temp_Seca_Max_Dia_Prom']])
datos['Temp_Seca_Max_Dia_Prom'] = knnImputer.transform(datos[['Temp_Seca_Max_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Seca_Max_Sem']])
datos['Temp_Seca_Max_Sem'] = knnImputer.transform(datos[['Temp_Seca_Max_Sem']]).ravel()
knnImputer.fit(datos[['Temp_Seca_Min_Dia_Prom']])
datos['Temp_Seca_Min_Dia_Prom'] = knnImputer.transform(datos[['Temp_Seca_Min_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Seca_Min_Sem']])
datos['Temp_Seca_Min_Sem'] = knnImputer.transform(datos[['Temp_Seca_Min_Sem']]).ravel()
knnImputer.fit(datos[['Prom_Dia_Preci']])
datos['Prom_Dia_Preci'] = knnImputer.transform(datos[['Prom_Dia_Preci']]).ravel()
knnImputer.fit(datos[['Total_Preci_Sem']])
datos['Total_Preci_Sem'] = knnImputer.transform(datos[['Total_Preci_Sem']]).ravel()
knnImputer.fit(datos[['Temp_Max_Dia_Prom']])
datos['Temp_Max_Dia_Prom'] = knnImputer.transform(datos[['Temp_Max_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Max_Sem']])
datos['Temp_Max_Sem'] = knnImputer.transform(datos[['Temp_Max_Sem']]).ravel()
knnImputer.fit(datos[['Temp_Min_Dia_Prom']])
datos['Temp_Min_Dia_Prom'] = knnImputer.transform(datos[['Temp_Min_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Min_Sem']])
datos['Temp_Min_Sem'] = knnImputer.transform(datos[['Temp_Min_Sem']]).ravel()
knnImputer.fit(datos[['Hum_Rel_Max']])
datos['Hum_Rel_Max'] = knnImputer.transform(datos[['Hum_Rel_Max']]).ravel()
knnImputer.fit(datos[['Hum_Rel_Min']])
datos['Hum_Rel_Min'] = knnImputer.transform(datos[['Hum_Rel_Min']]).ravel()


datos.shape

(364, 13)

In [4]:
train_size = int(len(datos) * 0.8)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

291 73


In [5]:
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_columns = ['Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem', 'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
             'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
             'Temp_Min_Dia_Prom', 'Temp_Min_Sem', 'Hum_Rel_Max', 'Hum_Rel_Min']

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train.head()

,Casos_Dengue,Temp_Seca_Max_Dia_Prom,Temp_Seca_Max_Sem,Temp_Seca_Min_Dia_Prom,Temp_Seca_Min_Sem,Prom_Dia_Preci,Total_Preci_Sem,Temp_Max_Dia_Prom,Temp_Max_Sem,Temp_Min_Dia_Prom,Temp_Min_Sem,Hum_Rel_Max,Hum_Rel_Min
0,23,0.636364,0.551724,0.672414,0.542857,0.020805,0.021093,0.457627,0.466667,0.673077,0.704545,0.605882,0.283582
1,25,0.563636,0.310345,0.413793,0.285714,0.000000,0.000000,0.389831,0.333333,0.423077,0.454545,0.664706,0.279851
2,21,0.545455,0.206897,0.637931,0.600000,0.187919,0.187919,0.474576,0.400000,0.730769,0.750000,0.688235,0.227612
3,30,0.472727,0.241379,0.689655,0.514286,0.110067,0.110259,0.305085,0.233333,0.730769,0.659091,0.647059,0.279851
4,22,0.618182,0.413793,0.706897,0.714286,0.089933,0.090125,0.406780,0.366667,0.769231,0.818182,0.682353,0.317164


In [6]:
train.describe()

,Casos_Dengue,Temp_Seca_Max_Dia_Prom,Temp_Seca_Max_Sem,Temp_Seca_Min_Dia_Prom,Temp_Seca_Min_Sem,Prom_Dia_Preci,Total_Preci_Sem,Temp_Max_Dia_Prom,Temp_Max_Sem,Temp_Min_Dia_Prom,Temp_Min_Sem,Hum_Rel_Max,Hum_Rel_Min
count,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000
mean,10.773196,0.499758,0.404458,0.696437,0.719780,0.154395,0.154177,0.427441,0.456603,0.703062,0.772476,0.551810,0.448271
std,9.852889,0.198603,0.187588,0.128674,0.124133,0.191217,0.191165,0.182036,0.191996,0.136331,0.119068,0.191008,0.193574
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.345455,0.275862,0.637931,0.657143,0.023826,0.024928,0.288136,0.316667,0.634615,0.727273,0.435294,0.313433
50%,7.000000,0.509091,0.401498,0.706897,0.742857,0.089933,0.090125,0.427302,0.433333,0.711538,0.795455,0.564706,0.439374
75%,17.000000,0.636364,0.551724,0.758621,0.800000,0.216779,0.216683,0.559322,0.600000,0.769231,0.818182,0.682353,0.574627
max,41.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
train['Casos_Dengue'].head()

0    0.560976
1    0.609756
2    0.512195
3    0.731707
4    0.536585
Name: Casos_Dengue, dtype: float64

In [8]:
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test.head()

,Casos_Dengue,Temp_Seca_Max_Dia_Prom,Temp_Seca_Max_Sem,Temp_Seca_Min_Dia_Prom,Temp_Seca_Min_Sem,Prom_Dia_Preci,Total_Preci_Sem,Temp_Max_Dia_Prom,Temp_Max_Sem,Temp_Min_Dia_Prom,Temp_Min_Sem,Hum_Rel_Max,Hum_Rel_Min
291,18,0.727273,0.655172,0.741379,0.771429,0.481208,0.481304,0.644068,0.633333,0.730769,0.795455,0.229412,0.201493
292,15,0.200000,0.275862,0.620690,0.714286,0.208725,0.209012,0.305085,0.366667,0.653846,0.772727,0.658824,0.641791
293,9,0.654545,0.551724,0.620690,0.600000,0.026174,0.025887,0.627119,0.600000,0.673077,0.704545,0.352941,0.302239
294,12,0.690909,0.689655,0.637931,0.742857,0.434228,0.434324,0.559322,0.766667,0.692308,0.818182,0.505882,0.305970
295,20,0.781818,0.620690,0.603448,0.714286,0.009396,0.009588,0.694915,0.700000,0.615385,0.704545,0.347059,0.152985


In [9]:
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])
test['Casos_Dengue'].head()

291    0.439024
292    0.365854
293    0.219512
294    0.292683
295    0.487805
Name: Casos_Dengue, dtype: float64

In [10]:
def create_dataset(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [11]:
time_steps = 7
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(284, 7, 13) (284,) (66, 7, 13) (66,)


In [12]:
from tensorflow.keras.layers import InputLayer

def LSTM_model(neuronas,
               dropout_rate,
               num_layers,
               activation = 'relu',
               init_mode='uniform',
              ):
    
    model = keras.Sequential()
    
    # CAPA DE ENTRADA
    model.add(keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
    
    
    #CAPAS INTERMEDIAS
    for i in range(num_layers):
        model.add(keras.layers.LSTM(units=neuronas,
                                    activation=activation,
                                    return_sequences = True,
                                    dropout=dropout_rate,
                                    kernel_initializer=init_mode))
        
        
    #CAPA DE SALIDA
    model.add(keras.layers.Dense(1, activation="linear"))
    
    #COMPILACIÓN DEL MODELO
    model.compile(optimizer='adam', loss = 'mse', metrics = ['mse'])
    
    return model

In [13]:
#HIPERPARÁMETROS

neuronas = [50, 70, 100]
activation = ['sigmoid','relu', 'tanh']
init_mode = ['uniform', 'normal', 'zero']
num_layers = [3, 5, 7]
dropout_rate = [0.1, 0.3, 0.5]

#optimizer = ['adam', 'sgd', 'rmsprop']
#batch_size = [40, 60, 80]
#epochs = [10, 50, 100]
#momentum = [0.2, 0.4, 0.5]

param_grid = dict(model__neuronas=neuronas,
                  model__activation=activation,
                  model__init_mode=init_mode,
                  model__dropout_rate=dropout_rate,
                  model__num_layers=num_layers)
                  #batch_size=batch_size,
                  #epochs=epochs,
                  #optimizer__momentum=momentum

In [14]:
model =  KerasRegressor(model=LSTM_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=5, n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [15]:
fitted_model = rs.fit(X_train, y_train)

In [16]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__num_layers,param_model__neuronas,param_model__init_mode,param_model__dropout_rate,param_model__activation,mean_test_score,std_test_score,mean_train_score,std_train_score
0,3,100,uniform,0.5,tanh,NaN,NaN,NaN,NaN
1,5,50,zero,0.1,sigmoid,NaN,NaN,NaN,NaN
2,3,100,normal,0.5,relu,NaN,NaN,NaN,NaN
3,5,50,uniform,0.3,tanh,NaN,NaN,NaN,NaN
4,5,70,uniform,0.3,sigmoid,NaN,NaN,NaN,NaN
5,5,50,uniform,0.1,tanh,NaN,NaN,NaN,NaN
6,7,70,uniform,0.3,tanh,NaN,NaN,NaN,NaN
7,7,100,zero,0.5,sigmoid,NaN,NaN,NaN,NaN
8,5,50,normal,0.1,relu,NaN,NaN,NaN,NaN
9,3,100,zero,0.1,sigmoid,NaN,NaN,NaN,NaN


In [17]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: nan using {'model__num_layers': 3, 'model__neuronas': 100, 'model__init_mode': 'uniform', 'model__dropout_rate': 0.5, 'model__activation': 'tanh'}
